In [0]:
df_infer = spark.read.option("header", True).option("inferSchema", True) \
    .csv("dbfs:/FileStore/course_enrollments.csv")
df_infer.show()
df_infer.printSchema()


+------------+-----------+--------------------+-----------+----------+---------------+------+---------+
|EnrollmentID|StudentName|          CourseName|   Category|EnrollDate|ProgressPercent|Rating|   Status|
+------------+-----------+--------------------+-----------+----------+---------------+------+---------+
|      ENR001|     Aditya|Python for Beginners|Programming|2024-05-10|             80|   4.5|   Active|
|      ENR002|     Simran|Data Analysis wit...|  Analytics|2024-05-12|            100|   4.7|Completed|
|      ENR003|     Aakash| Power BI Essentials|  Analytics|2024-05-13|             30|   3.8|   Active|
|      ENR004|       Neha|         Java Basics|Programming|2024-05-15|              0|  NULL| Inactive|
|      ENR005|       Zara|Machine Learning 101|         AI|2024-05-17|             60|   4.2|   Active|
|      ENR006|    Ibrahim|Python for Beginners|Programming|2024-05-18|             90|   4.6|Completed|
+------------+-----------+--------------------+-----------+-----

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, DateType

schema_manual = StructType([
    StructField("EnrollmentID", StringType(), True),
    StructField("StudentName", StringType(), True),
    StructField("CourseName", StringType(), True),
    StructField("Category", StringType(), True),
    StructField("EnrollDate", DateType(), True),
    StructField("ProgressPercent", IntegerType(), True),
    StructField("Rating", DoubleType(), True),
    StructField("Status", StringType(), True)
])

df_manual = spark.read.option("header", True).schema(schema_manual) \
    .csv("dbfs:/FileStore/course_enrollments.csv")
df_manual.show()
df_manual.printSchema()


+------------+-----------+--------------------+-----------+----------+---------------+------+---------+
|EnrollmentID|StudentName|          CourseName|   Category|EnrollDate|ProgressPercent|Rating|   Status|
+------------+-----------+--------------------+-----------+----------+---------------+------+---------+
|      ENR001|     Aditya|Python for Beginners|Programming|2024-05-10|             80|   4.5|   Active|
|      ENR002|     Simran|Data Analysis wit...|  Analytics|2024-05-12|            100|   4.7|Completed|
|      ENR003|     Aakash| Power BI Essentials|  Analytics|2024-05-13|             30|   3.8|   Active|
|      ENR004|       Neha|         Java Basics|Programming|2024-05-15|              0|  NULL| Inactive|
|      ENR005|       Zara|Machine Learning 101|         AI|2024-05-17|             60|   4.2|   Active|
|      ENR006|    Ibrahim|Python for Beginners|Programming|2024-05-18|             90|   4.6|Completed|
+------------+-----------+--------------------+-----------+-----

In [0]:
df_filtered = df_manual.filter(df_manual.ProgressPercent < 50)
df_filtered.show()


+------------+-----------+-------------------+-----------+----------+---------------+------+--------+
|EnrollmentID|StudentName|         CourseName|   Category|EnrollDate|ProgressPercent|Rating|  Status|
+------------+-----------+-------------------+-----------+----------+---------------+------+--------+
|      ENR003|     Aakash|Power BI Essentials|  Analytics|2024-05-13|             30|   3.8|  Active|
|      ENR004|       Neha|        Java Basics|Programming|2024-05-15|              0|  NULL|Inactive|
+------------+-----------+-------------------+-----------+----------+---------------+------+--------+



In [0]:
from pyspark.sql.functions import avg, when, col

avg_rating = df_manual.select(avg("Rating")).first()[0]

df_filled = df_manual.withColumn("Rating",
    when(col("Rating").isNull(), avg_rating).otherwise(col("Rating")))
df_filled.show()


+------------+-----------+--------------------+-----------+----------+---------------+-----------------+---------+
|EnrollmentID|StudentName|          CourseName|   Category|EnrollDate|ProgressPercent|           Rating|   Status|
+------------+-----------+--------------------+-----------+----------+---------------+-----------------+---------+
|      ENR001|     Aditya|Python for Beginners|Programming|2024-05-10|             80|              4.5|   Active|
|      ENR002|     Simran|Data Analysis wit...|  Analytics|2024-05-12|            100|              4.7|Completed|
|      ENR003|     Aakash| Power BI Essentials|  Analytics|2024-05-13|             30|              3.8|   Active|
|      ENR004|       Neha|         Java Basics|Programming|2024-05-15|              0|4.359999999999999| Inactive|
|      ENR005|       Zara|Machine Learning 101|         AI|2024-05-17|             60|              4.2|   Active|
|      ENR006|    Ibrahim|Python for Beginners|Programming|2024-05-18|          

In [0]:
df_status = df_filled.withColumn("IsActive",
    when(col("Status") == "Active", 1).otherwise(0))
df_status.show()


+------------+-----------+--------------------+-----------+----------+---------------+-----------------+---------+--------+
|EnrollmentID|StudentName|          CourseName|   Category|EnrollDate|ProgressPercent|           Rating|   Status|IsActive|
+------------+-----------+--------------------+-----------+----------+---------------+-----------------+---------+--------+
|      ENR001|     Aditya|Python for Beginners|Programming|2024-05-10|             80|              4.5|   Active|       1|
|      ENR002|     Simran|Data Analysis wit...|  Analytics|2024-05-12|            100|              4.7|Completed|       0|
|      ENR003|     Aakash| Power BI Essentials|  Analytics|2024-05-13|             30|              3.8|   Active|       1|
|      ENR004|       Neha|         Java Basics|Programming|2024-05-15|              0|4.359999999999999| Inactive|       0|
|      ENR005|       Zara|Machine Learning 101|         AI|2024-05-17|             60|              4.2|   Active|       1|
|      E

In [0]:
df_status.groupBy("CourseName").agg(avg("ProgressPercent").alias("AvgProgress")).show()


+--------------------+-----------+
|          CourseName|AvgProgress|
+--------------------+-----------+
|Data Analysis wit...|      100.0|
|         Java Basics|        0.0|
|Machine Learning 101|       60.0|
|Python for Beginners|       85.0|
| Power BI Essentials|       30.0|
+--------------------+-----------+



In [0]:
df_status.groupBy("Category").count().show()


+-----------+-----+
|   Category|count|
+-----------+-----+
|Programming|    3|
|         AI|    1|
|  Analytics|    2|
+-----------+-----+



In [0]:
from pyspark.sql.functions import desc

df_status.groupBy("CourseName").count().orderBy(desc("count")).limit(1).show()


+--------------------+-----+
|          CourseName|count|
+--------------------+-----+
|Python for Beginners|    2|
+--------------------+-----+



In [0]:
df_course_details = spark.read.option("header", True) \
    .csv("dbfs:/FileStore/course_details.csv")
df_course_details.show()


+--------------------+-------------+----------+
|          CourseName|DurationWeeks|Instructor|
+--------------------+-------------+----------+
|Python for Beginners|            4|    Rakesh|
|Data Analysis wit...|            3|    Anjali|
| Power BI Essentials|            5|     Rekha|
|         Java Basics|            6|     Manoj|
|Machine Learning 101|            8|     Samir|
+--------------------+-------------+----------+



In [0]:
df_joined = df_status.join(df_course_details, on="CourseName", how="left")
df_joined.show()


+--------------------+------------+-----------+-----------+----------+---------------+-----------------+---------+--------+-------------+----------+
|          CourseName|EnrollmentID|StudentName|   Category|EnrollDate|ProgressPercent|           Rating|   Status|IsActive|DurationWeeks|Instructor|
+--------------------+------------+-----------+-----------+----------+---------------+-----------------+---------+--------+-------------+----------+
|Python for Beginners|      ENR001|     Aditya|Programming|2024-05-10|             80|              4.5|   Active|       1|            4|    Rakesh|
|Data Analysis wit...|      ENR002|     Simran|  Analytics|2024-05-12|            100|              4.7|Completed|       0|            3|    Anjali|
| Power BI Essentials|      ENR003|     Aakash|  Analytics|2024-05-13|             30|              3.8|   Active|       1|            5|     Rekha|
|         Java Basics|      ENR004|       Neha|Programming|2024-05-15|              0|4.359999999999999| I

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import rank

window_spec = Window.partitionBy("CourseName").orderBy(col("ProgressPercent").desc())

df_ranked = df_joined.withColumn("Rank", rank().over(window_spec))
df_ranked.select("StudentName", "CourseName", "ProgressPercent", "Rank").show()


+-----------+--------------------+---------------+----+
|StudentName|          CourseName|ProgressPercent|Rank|
+-----------+--------------------+---------------+----+
|     Simran|Data Analysis wit...|            100|   1|
|       Neha|         Java Basics|              0|   1|
|       Zara|Machine Learning 101|             60|   1|
|     Aakash| Power BI Essentials|             30|   1|
|    Ibrahim|Python for Beginners|             90|   1|
|     Aditya|Python for Beginners|             80|   2|
+-----------+--------------------+---------------+----+



In [0]:
from pyspark.sql.functions import lead, lag

window_spec_date = Window.partitionBy("Category").orderBy("EnrollDate")

df_dates = df_joined.withColumn("NextEnrollDate", lead("EnrollDate").over(window_spec_date)) \
    .withColumn("PrevEnrollDate", lag("EnrollDate").over(window_spec_date))

df_dates.select("StudentName", "Category", "EnrollDate", "PrevEnrollDate", "NextEnrollDate").show()


+-----------+-----------+----------+--------------+--------------+
|StudentName|   Category|EnrollDate|PrevEnrollDate|NextEnrollDate|
+-----------+-----------+----------+--------------+--------------+
|       Zara|         AI|2024-05-17|          NULL|          NULL|
|     Simran|  Analytics|2024-05-12|          NULL|    2024-05-13|
|     Aakash|  Analytics|2024-05-13|    2024-05-12|          NULL|
|     Aditya|Programming|2024-05-10|          NULL|    2024-05-15|
|       Neha|Programming|2024-05-15|    2024-05-10|    2024-05-18|
|    Ibrahim|Programming|2024-05-18|    2024-05-15|          NULL|
+-----------+-----------+----------+--------------+--------------+



In [0]:
df_pivot = df_joined.groupBy("Category").pivot("Status").count()
df_pivot.show()


+-----------+------+---------+--------+
|   Category|Active|Completed|Inactive|
+-----------+------+---------+--------+
|Programming|     1|        1|       1|
|         AI|     1|     NULL|    NULL|
|  Analytics|     1|        1|    NULL|
+-----------+------+---------+--------+



In [0]:
from pyspark.sql.functions import year, month

df_date_parts = df_joined.withColumn("EnrollYear", year("EnrollDate")) \
                         .withColumn("EnrollMonth", month("EnrollDate"))
df_date_parts.select("EnrollDate", "EnrollYear", "EnrollMonth").show()


+----------+----------+-----------+
|EnrollDate|EnrollYear|EnrollMonth|
+----------+----------+-----------+
|2024-05-10|      2024|          5|
|2024-05-12|      2024|          5|
|2024-05-13|      2024|          5|
|2024-05-15|      2024|          5|
|2024-05-17|      2024|          5|
|2024-05-18|      2024|          5|
+----------+----------+-----------+



In [0]:
df_cleaned = df_date_parts.filter((col("Status").isNotNull()) & (col("Status") != ""))


In [0]:
df_dedup = df_cleaned.dropDuplicates(["EnrollmentID"])


In [0]:
# CSV
df_dedup.write.mode("overwrite").option("header", True) \
    .csv("dbfs:/FileStore/output/enrollments_csv")

# JSON
df_dedup.write.mode("overwrite").json("dbfs:/FileStore/output/enrollments_json")

# Parquet with snappy compression
df_dedup.write.mode("overwrite").option("compression", "snappy") \
    .parquet("dbfs:/FileStore/output/enrollments_parquet")
